# Mirascope Quickstart Guide

1. [Setup](#Setup)
2. [Prompt Templates](#Prompt-Templates)
3. [Basic LLM Call](#Basic-LLM-Call)
4. [Streaming Responses](#Streaming-Responses)
5. [Response Models](#Response-Models)
6. [Asynchronous Processing](#Asynchronous-Processing)
7. [JSON Mode](#JSON-Mode)
8. [Output Parsers](#Output-Parsers)

## Setup

Mirascope supports various LLM providers, including [OpenAI](https://openai.com/), [Anthropic](https://www.anthropic.com/), [Mistral](https://mistral.ai/), [Gemini](https://gemini.google.com), [Groq](https://groq.com/), [Cohere](https://cohere.com/), [LiteLLM](https://www.litellm.ai/), [Azure AI](https://azure.microsoft.com/en-us/solutions/ai), and [Vertex AI](https://cloud.google.com/vertex-ai). For the purposes of this guide, we will be using OpenAI. Let's start by installing Mirascope and its dependencies:

In [1]:
!pip install "mirascope[openai]"

This command installs Mirascope along with the necessary packages for the OpenAI integration.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
# Set the appropriate API key for the provider you're using

## Prompt Templates

Prompt templates in Mirascope allow you to create dynamic and reusable prompts.

In [4]:
from mirascope.core import prompt_template


@prompt_template("What is the capital of {country}?")
def get_capital_prompt(country: str): ...


print(get_capital_prompt("Japan"))
# [BaseMessageParam(role='user', content='What is the capital of Japan?')]

[BaseMessageParam(role='user', content='What is the capital of Japan?')]


In this example:
1. We import the `prompt_template` decorator from Mirascope.
2. We define a function `get_capital_prompt` and decorate it with `@prompt_template`.
3. The `{country}` placeholder in the template will be replaced with the actual argument passed to the function.
4. When we call `get_capital_prompt("Japan")`, it returns a list containing a `BaseMessageParam` object with the role 'user' and the content "What is the capital of Japan?"

Prompt templates are particularly useful for creating consistent prompts across multiple calls and for easily modifying prompt structures. Here's a more complex example:

In [ ]:
@prompt_template(
    """
    SYSTEM:
    You are a knowledgeable assistant specializing in world geography.
    Your task is to provide detailed information about the capital cities of various countries.
    
    USER:
    Please provide the following information about the capital of {country}:
    1. The name of the capital city
    2. Its population (approximate is fine)
    3. Three major landmarks or attractions
    """
)
def get_capital_info(country: str): ...


print(get_capital_info("France"))
# [BaseMessageParam(role='system', content='You are a knowledgeable assistant specializing in world geography.\nYour task is to provide detailed information about the capital cities of various countries.'),
#  BaseMessageParam(role='user', content='Please provide the following information about the capital of France:\n1. The name of the capital city\n2. Its population (approximate is fine)\n3. Three major landmarks or attractions')]

This more advanced example demonstrates:
1. A multi-line prompt with distinct SYSTEM and USER roles, useful for chat-based LLMs.
2. Structured queries within the prompt, allowing for more detailed and specific responses.
3. Proper indentation and formatting for readability and correct parsing.
4. The function now returns a list of `BaseMessageParam` objects, one for each role (SYSTEM and USER) in the prompt.

Prompt templates improve code readability, maintain consistency, and facilitate experimentation with different prompt structures. They also provide a structured way to work with different message roles in chat-based LLM interactions.

For more advanced techniques, including handling multiple variables and conditional prompts, refer to our [documentation on Prompts](https://docs.mirascope.io/learn/prompts).

## Basic LLM Call

The `call` decorator in Mirascope transforms functions with prompt templates into LLM API calls. This allows you to seamlessly integrate LLM interactions into your Python code.

In [47]:
from mirascope.core import openai


@openai.call("gpt-4o-mini")
@prompt_template("What is the capital of {country}?")
def get_capital(country: str): ...


response = get_capital("Japan")
print(response.content)
# The capital of Japan is Tokyo.

The capital of Japan is Tokyo.


In this example:
1. We import the `openai` module from Mirascope, which provides the `call` decorator.
2. The `@openai.call("gpt-4o-mini")` decorator specifies which OpenAI model to use.
3. We combine the `call` decorator with our previous `prompt_template`.
4. The `...` in the function body indicates that we don't need to implement the function ourselves; Mirascope handles the LLM interaction.
5. When we call `get_capital("Japan")`, it sends a request to the OpenAI API and returns the response.
6. We print the `content` of the response, which contains the LLM's answer.

This approach allows you to use LLMs as if they were regular Python functions, making it easy to integrate AI capabilities into your applications. For more advanced usage, including controlling model parameters and handling errors, see our [documentation on Calls](https://docs.mirascope.io/learn/calls).

## Streaming Responses

Streaming allows you to process LLM responses in real-time, which is particularly useful for long-form content generation or when you want to provide immediate feedback to users.

In [48]:
@openai.call("gpt-4o-mini", stream=True)
@prompt_template("Provide a brief description of {city}.")
def stream_city_info(city: str): ...


for chunk, _ in stream_city_info("Tokyo"):
    print(chunk.content, end="", flush=True)
# Tokyo, the capital of Japan, is a vibrant and bustling metropolis known for its unique blend of traditional culture and modern innovation. With a population of over 13 million people, it is one of the most populous cities in the world. Tokyo is famous for its towering skyscrapers, historic temples, and diverse neighborhoods, ranging from the fashion-forward streets of Harajuku to the historic Asakusa district. The city offers a rich culinary scene, world-class shopping, and a lively arts and entertainment scene. Its efficient public transportation system makes it easy to explore, while its blend of old and new provides a captivating experience for both residents and visito

Tokyo, the capital of Japan, is a vibrant and bustling metropolis known for its unique blend of traditional culture and modern innovation. With a population of over 13 million people, it is one of the most populous cities in the world. Tokyo is famous for its towering skyscrapers, historic temples, and diverse neighborhoods, ranging from the fashion-forward streets of Harajuku to the historic Asakusa district. The city offers a rich culinary scene, world-class shopping, and a lively arts and entertainment scene. Its efficient public transportation system makes it easy to explore, while its blend of old and new provides a captivating experience for both residents and visitors.

Here's what's happening in this streaming example:
1. We use the `stream=True` parameter in the `@openai.call` decorator to enable streaming.
2. The function returns an iterator that yields chunks of the response as they become available.
3. We iterate over the chunks, printing each one immediately.
4. The `end=""` and `flush=True` parameters in the print function ensure that the output is displayed in real-time without line breaks.

Streaming is beneficial for:
- Providing immediate feedback to users
- Processing very long responses efficiently
- Implementing typewriter-like effects in user interfaces

For more advanced streaming techniques, including error handling and processing streamed content, refer to our [documentation on Streams](https://docs.mirascope.io/learn/streams).

## Response Models

Response models in Mirascope allow you to structure and validate the output from LLMs. This feature is particularly useful when you need to ensure that the LLM's response adheres to a specific format or contains certain fields.

In [49]:
from pydantic import BaseModel


class Capital(BaseModel):
    city: str
    country: str


@openai.call("gpt-4o-mini", response_model=Capital)
@prompt_template("{query}")
def extract_capital(query: str): ...


capital = extract_capital("The capital of France is Paris")
print(capital)
# city='Paris' country='France'

city='Paris' country='France'


For more details on response models, including advanced validation techniques, check our [documentation on Response Models](https://docs.mirascope.io/learn/response-models).

## Asynchronous Processing

Mirascope supports asynchronous processing, allowing for efficient parallel execution of multiple LLM calls. This is particularly useful when you need to make many LLM calls concurrently or when working with asynchronous web frameworks.

In [50]:
import asyncio


@openai.call("gpt-4o-mini", response_model=Capital)
@prompt_template("What is the capital of {country}?")
async def get_capital_async(country: str): ...


async def main():
    countries = ["France", "Japan", "Brazil"]
    tasks = [get_capital_async(country) for country in countries]
    capitals = await asyncio.gather(*tasks)
    for capital in capitals:
        print(f"The capital of {capital.country} is {capital.city}")


# await main() when running in a Jupyter notebook
await main()

# The capital of France is Paris
# The capital of Japan is Tokyo
# The capital of Brazil is Brazil

# asyncio.run(main()) when running in a Python script
# asyncio.run(main())

The capital of France is Paris
The capital of Japan is Tokyo
The capital of Brazil is Brazil


This asynchronous example demonstrates:
1. An async version of our `get_capital` function, defined with `async def`.
2. Use of `asyncio.gather()` to run multiple async tasks concurrently.
3. Processing of results as they become available.

Asynchronous processing offers several advantages:
- Improved performance when making multiple LLM calls
- Better resource utilization
- Compatibility with async web frameworks like FastAPI or aiohttp

For more advanced asynchronous techniques, including error handling and async streaming, refer to our [documentation on Async](https://docs.mirascope.io/learn/async).

## JSON Mode

JSON mode allows you to directly parse LLM outputs as JSON. This is particularly useful when you need structured data from your LLM calls.

In [51]:
@openai.call("gpt-4o-mini", json_mode=True)
@prompt_template("Provide information about {city} in JSON format")
def city_info(city: str): ...


response = city_info("Tokyo")
print(response.content)  # This will be a JSON-formatted string
# {
#  "city": "Tokyo",
#  "country": "Japan",
#  "population": 13929286,
#  "area_km2": 2191,
#  "famous_landmarks": [
#    {
#      "name": "Tokyo Tower",
#      "type": "Observation Tower",
#      "height_m": 333
#    },
#    {
#      "name": "Senso-ji",
#      "type": "Temple",
#      "region": "Asakusa"
#    },
#    {
#      "name": "Shibuya Crossing",
#      "type": "Intersection",
#      "description": "Famous pedestrian scramble"
#    },
#    {
#      "name": "Tokyo Skytree",
#      "type": "Broadcasting Tower",
#      "height_m": 634
#    }
#  ],
#  "public_transportation": {
#    "subway_lines": 13,
#    "bus_routes": 1000,
#    "railway_network": "Tokyo Metro and JR East"
#  },
#  "cuisine": [
#    {
#      "name": "Sushi",
#      "description": "Vinegared rice with raw fish"
#    },
#    {
#      "name": "Ramen",
#      "description": "Japanese noodle soup"
#    },
#    {
#      "name": "Tempura",
#      "description": "Batter-fried seafood and vegetables"
#    }
#  ]
# }

{
  "city": "Tokyo",
  "country": "Japan",
  "population": 13929286,
  "area_km2": 2191,
  "famous_landmarks": [
    {
      "name": "Tokyo Tower",
      "type": "Observation Tower",
      "height_m": 333
    },
    {
      "name": "Senso-ji",
      "type": "Temple",
      "region": "Asakusa"
    },
    {
      "name": "Shibuya Crossing",
      "type": "Intersection",
      "description": "Famous pedestrian scramble"
    },
    {
      "name": "Tokyo Skytree",
      "type": "Broadcasting Tower",
      "height_m": 634
    }
  ],
  "public_transportation": {
    "subway_lines": 13,
    "bus_routes": 1000,
    "railway_network": "Tokyo Metro and JR East"
  },
  "cuisine": [
    {
      "name": "Sushi",
      "description": "Vinegared rice with raw fish"
    },
    {
      "name": "Ramen",
      "description": "Japanese noodle soup"
    },
    {
      "name": "Tempura",
      "description": "Batter-fried seafood and vegetables"
    }
  ]
}


In this example:
1. We use `json_mode=True` in the `@openai.call` decorator to instruct the LLM to return JSON.
2. The LLM's response will be a valid JSON string, which can be easily parsed into a Python dictionary or object.

JSON mode is beneficial for:
- Ensuring structured outputs from LLMs
- Easy integration with data processing pipelines
- Creating APIs that return JSON data

Note that not all providers have an explicit JSON mode. For those providers, we attempt to instruct the model to provide JSON; however, there is no guarantee that it will output only JSON (it may start with some text like "Here is the JSON: ..."). This is where Output Parsers can be useful.

For more information on JSON mode and its limitations with different providers, refer to our [documentation on JSON Mode](https://docs.mirascope.io/learn/json-mode).

## Output Parsers

Output parsers allow you to process LLM responses in custom formats. They are particularly useful when working with JSON outputs, especially for providers like Anthropic that don't have a strict JSON mode.

In [ ]:
!pip install "mirascope[anthropic]"

In [52]:
import json

from mirascope.core import anthropic


def only_json(response: anthropic.AnthropicCallResponse) -> str:
    json_start = response.content.index("{")
    json_end = response.content.rfind("}")
    return response.content[json_start : json_end + 1]


@anthropic.call("claude-3-5-sonnet-20240620", json_mode=True, output_parser=only_json)
@prompt_template("Extract {fields} from the following text: {text}")
def json_extraction(text: str, fields: list[str]): ...


json_response = json_extraction(
    text="The capital of France is Paris",
    fields=["capital", "country"],
)
print(json.loads(json_response))
# {'capital': 'Paris', 'country': 'France'}

{'capital': 'Paris', 'country': 'France'}


In this example:
1. We define a custom `only_json` parser that extracts the JSON portion from the response.
2. We use both `json_mode=True` and the custom output parser to ensure we get clean JSON output.
3. The `json_extraction` function demonstrates how to combine JSON mode with a custom parser.

Output parsers are useful for:
- Extracting specific formats or data structures from LLM responses
- Cleaning and standardizing LLM outputs
- Implementing custom post-processing logic

For more information on output parsers and advanced usage scenarios, see our [documentation on Output Parsers](https://docs.mirascope.io/learn/output-parsers).

This concludes our Quickstart Guide to Mirascope. We've covered the main features of the library, including prompt templates, basic calls, streaming, response models, asynchronous processing, JSON mode, and output parsers. Each of these features can be combined and customized to create powerful, flexible AI applications.

For more detailed information on each of these topics and advanced usage, including dynamic configuration and chaining, please refer to our comprehensive [Learn documentation](https://docs.mirascope.io/learn).